In [ ]:
#pip install opencv-python

In [1]:
#from PIL import Image
import cv2 as cv
import os
import sys
import numpy as np
import json

Directory

In [2]:
#root_dir = os.environ.get('GITLAB_ROOT', None)
directory = os.path.dirname("C:/Users/morit/OneDrive/UNI/Master/WS22/APP-RAS/Programming/data/sample_trainingsdata/")
directory

'C:/Users/morit/OneDrive/UNI/Master/WS22/APP-RAS/Programming/data/sample_trainingsdata'

Preprocess data

In [3]:
rgb_liste = list()
#test = np.zeros(shape = (51,160,960,3))

rgb_dir=directory+"/rgb/"

for subdir, dirs, files in os.walk(rgb_dir):
    for f in files:
        img = cv.imread(rgb_dir+f,cv.IMREAD_COLOR) # Greyscale
        img_norm = cv.normalize(img,  None, 0, 1, cv.NORM_MINMAX, dtype=cv.CV_32F)

        
        rgb_liste.append(img_norm)
np.shape(rgb_liste)

(53, 160, 960, 3)

In [ ]:
# TODO research on normalization
#norm_img = np.zeros((160,960))
#cv.normalize(test,  None, 0, 1, cv.NORM_MINMAX, dtype=cv2.CV_32F)

In [4]:
measurements_liste = list()

vc_dir=directory+"/measurements/"

z = 0
for subdir, dirs, files in os.walk(vc_dir):
    #print(files)
    for f in files:
        #print(z)
        z+=1
        #print(f)
        file_name = vc_dir+f

        with open(file_name, 'r') as f:
            measurements = json.load(f)
        
        #dictionary = {"steer":measurements["steer"],"throttle":measurements["throttle"],"brake":measurements["brake"]}
        array = np.array([measurements["steer"],measurements["throttle"],measurements["brake"]])
        
        measurements_liste.append(array)
np.shape(measurements_liste)

(53, 3)

models

In [5]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D 

In [6]:
#pip install tensorflow
#pip install keras

In [6]:
#rename
X = np.asarray(rgb_liste)
print(np.shape(X))
print(type(X))
Y = np.asarray(measurements_liste)
print(type(Y))
print(np.shape(Y))

(53, 160, 960, 3)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(53, 3)


In [7]:
model = Sequential()

model.add(Conv2D(filters = 24,  input_shape=(160,960,3), kernel_size= (5,5), activation='relu'))
model.add(MaxPool2D(pool_size = (5,5)))

model.add(Conv2D(filters = 36, kernel_size= (5,5), activation='relu'))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(filters = 48, kernel_size= (2,2), activation='relu'))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(filters = 64, kernel_size= (2,2), activation='relu'))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(1000,activation="tanh"))
model.add(Dense(100,activation="tanh"))
model.add(Dense(50,activation="tanh"))
model.add(Dense(10,activation="tanh"))

model.add(Dense(3,activation="tanh"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 156, 956, 24)      1824      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 191, 24)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 187, 36)       21636     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 93, 36)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 92, 48)        6960      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 46, 48)        0

In [8]:
model.compile(loss='mse', optimizer='adam', metrics=['mse'])

In [10]:
model.fit(X, Y, epochs=5, batch_size=5)

Epoch 1/5
11/11 [==============================] - 5s 480ms/step - loss: 0.0367 - mse: 0.0367
Epoch 2/5
11/11 [==============================] - 5s 427ms/step - loss: 0.0383 - mse: 0.0383
Epoch 3/5
11/11 [==============================] - 5s 448ms/step - loss: 0.0350 - mse: 0.0350
Epoch 4/5
11/11 [==============================] - 5s 478ms/step - loss: 0.0315 - mse: 0.0315
Epoch 5/5
11/11 [==============================] - 5s 446ms/step - loss: 0.0302 - mse: 0.0302


In [11]:
preds = model.predict(X)

2/2 [==============================] - 1s 393ms/step


In [12]:
preds

array([[ 0.8651305 ,  0.6959351 , -0.14525102],
       [ 0.89290786,  0.6700313 ,  0.01170527],
       [ 0.86162406,  0.728245  , -0.13304362],
       [ 0.8747903 ,  0.72002167,  0.01343961],
       [ 0.8811029 ,  0.6954565 , -0.09331099],
       [ 0.8753763 ,  0.71308833, -0.10437395],
       [ 0.48870286,  0.6267498 ,  0.08001167],
       [ 0.00103457,  0.3308744 , -0.08891624],
       [-0.3193466 ,  0.67673385,  0.09002932],
       [-0.3161947 ,  0.6505206 ,  0.01314062],
       [ 0.01290829,  0.3640797 , -0.04907243],
       [ 0.09898337,  0.33642548, -0.0570787 ],
       [ 0.4433836 ,  0.6079135 , -0.09417185],
       [-0.02240603,  0.41482764, -0.01170446],
       [-0.03357119,  0.45627826, -0.0137134 ],
       [ 0.05938533,  0.7408738 ,  0.01834868],
       [-0.07225623,  0.47849068,  0.03525016],
       [ 0.21965721,  0.08671294, -0.00775516],
       [-0.14527525,  0.55279726,  0.0506571 ],
       [-0.02469465,  0.40625492,  0.02098302],
       [ 0.202409  ,  0.2522208 , -0.019

In [13]:
Y

array([[ 1.        ,  0.75      ,  0.        ],
       [ 1.        ,  0.75      ,  0.        ],
       [ 1.        ,  0.75      ,  0.        ],
       [ 1.        ,  0.75      ,  0.        ],
       [ 1.        ,  0.75      ,  0.        ],
       [ 1.        ,  0.75      ,  0.        ],
       [ 0.763     ,  0.75      ,  0.        ],
       [-0.156     ,  0.10045608,  0.        ],
       [-0.521     ,  0.75      ,  0.        ],
       [-0.608     ,  0.6692466 ,  0.        ],
       [-0.35      ,  0.05414442,  0.        ],
       [-0.006     ,  0.04743834,  0.        ],
       [ 0.177     ,  0.55628823,  0.        ],
       [ 0.207     ,  0.4905683 ,  0.        ],
       [ 0.126     ,  0.75      ,  0.        ],
       [ 0.019     ,  0.61852375,  0.        ],
       [-0.057     ,  0.75      ,  0.        ],
       [-0.071     ,  0.06669305,  0.        ],
       [-0.047     ,  0.75      ,  0.        ],
       [-0.009     ,  0.26315889,  0.        ],
       [ 0.016     ,  0.02535246,  0.   

Saving and loading Model TEST

In [14]:
model.save('C:/Users/morit/OneDrive/UNI/Master/WS22/APP-RAS/Programming/models/nvidia_cnn/model')

INFO:tensorflow:Assets written to: C:/Users/morit/OneDrive/UNI/Master/WS22/APP-RAS/Programming/models/nvidia_cnn/model\assets


INFO:tensorflow:Assets written to: C:/Users/morit/OneDrive/UNI/Master/WS22/APP-RAS/Programming/models/nvidia_cnn/model\assets


In [15]:
model = keras.models.load_model('C:/Users/morit/OneDrive/UNI/Master/WS22/APP-RAS/Programming/models/nvidia_cnn/model')

In [ ]:
preds = model.predict(X)
print(preds[1])